In [1]:
import zipfile
import pandas as pd

def read_zipped_csv(zipfilename, filename):
    """
    Open up a zipfile, and read out csv data from the speficied filename
    into a pandas dataframe.

    Parameters
    ----------
    zipfilename:
        Source zip file on disk.
    filename:
        File within the zip containing the data

    Returns
    -------
    A Pandas DataFrame.
    """
    # target path, wrapped in an object
    with zipfile.ZipFile(zipfilename) as archive:
        with archive.open(filename) as file:
            return pd.read_csv(file)

loans = read_zipped_csv('./lending-club-loan-data.zip', 'loan.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [2]:
import sqlite3
import zipfile
import pandas as pd

def read_zipped_sqlite(zipfilename, filename, query):
    """
    Open up a zipfile, and read out table data from the speficied filename
    into a pandas dataframe.

    Parameters
    ----------
    zipfilename:
        Source zip file on disk.
    filename:
        File within the zip containing the data
    tablename:
        Read this entire table.

    Returns
    -------
    A Pandas DataFrame.
    """
    # target path, wrapped in an object
    with zipfile.ZipFile(zipfilename) as archive:
        archive.extract(filename)
        with sqlite3.connect(filename) as connection:
            return pd.read_sql(query, connection)

loans = read_zipped_sqlite('./lending-club-loan-data.zip', 'database.sqlite', 'select * from loan')

In [3]:
loans.columns

Index(['index', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url',
       'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_sta

In [4]:
loans.iloc[0:1]

,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
